In [11]:
import numpy as np
import cvxpy as cp
import numpy as np
import pandas as pd
import time
from scipy.optimize import minimize
import random
from tqdm.notebook import tqdm

# 行動分類のラベル
large = ["必需行動", "拘束行動", "自由行動", "その他"] # 大分類
 
middle = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事関連", "学業", 
  "家事", "通勤", "通学", "社会参加", "会話・交際", "レジャー活動", 
  "マスメディア接触", "休息", "その他・不明"]
 # 中分類
 
small = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", "授業", "学校外の学習", 
  "炊事・掃除・洗濯", "買い物", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", 
  "スポーツ", "行楽・散策", "趣味・娯楽・教養 (インターネット除く)", "趣味・娯楽・教養のインターネット (動画除く)", 
  "インターネット動画", "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", "音楽", "休息", "その他", "不明"]
 # 小分類

def sort_df_by_list(df:pd.DataFrame,sort_column_by:str,sort_key_by:list):
  df = df.copy()  # 明示的にDataFrameのコピーを作成
  df.loc[:,"sort_order"] = df[sort_column_by].map({name: i for i,name in enumerate(sort_key_by)})
  df_sorted = df.sort_values('sort_order').drop('sort_order', axis=1)
  return df_sorted

In [5]:
"""国民生活時間調査のデータからの成型"""
# daily
daily_df = pd.read_csv('2020_4shihyo_danjonenso.csv')
daily_df = daily_df.set_axis(["Day","Group","Activity","Rate","AvgTimeAct","AvgTimeAll","StdDevAll"],axis=1)
daily_df["Rate"] = daily_df["Rate"]/100
daily_table = daily_df.query("Day == '平日' and Group == '男１０代'").query(f"Activity in @middle").filter(items=["Activity","Rate"])
# daily_table = daily_df.query("Day == '平日'").query(f"Activity in @middle").filter(items=["Activity","Rate"])
# time
time_df = pd.read_csv('2020_jikoku_danjonenso.csv')
time_df = time_df.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
time_df["Rate"] = time_df["Rate"]/100
time_table = time_df.query("Day == '平日' and Group == '男１０代'").query(f"Activity in @middle").filter(items=["Activity","Time","Rate"])
# time_table = time_df.query("Day == '平日'").query(f"Activity in @middle").filter(items=["Activity","Time","Rate"])
# get time list
time_list = list(time_table["Time"].unique())

orig_P_i = daily_table["Rate"].to_list() # オリジナルの一日の行為者率

In [6]:
time_table

,Activity,Time,Rate
0,睡眠,0:00,0.866
1,睡眠,0:15,0.872
2,睡眠,0:30,0.888
3,睡眠,0:45,0.888
4,睡眠,1:00,0.920
...,...,...,...
44443,その他・不明,22:45,0.048
44444,その他・不明,23:00,0.053
44445,その他・不明,23:15,0.064
44446,その他・不明,23:30,0.059


In [46]:
pd.set_option("display.max_columns",None)

In [57]:
df = pd.read_csv("C:/Users/tora2/downloads/persona_schedule.csv")
df

,曜日,時間,行為種別,詳細
0,月曜日,6:00 - 6:30,身のまわりの用事,目覚ましが鳴り、少し寝ぼけながら起床。今日は新しいプロジェクトのキックオフの日。軽いストレッ...
1,月曜日,6:30 - 7:00,朝食,子供たちを起こして、簡単な朝食を準備。目玉焼きとトースト、そしてフルーツ。夫と今日のスケジュ...
2,月曜日,7:00 - 8:30,通勤,電車に乗って通勤。途中でメールをチェックし、プロジェクトに関連する資料を再確認。少し緊張しつ...
3,月曜日,8:30 - 12:00,仕事関連,新プロジェクトのキックオフミーティング。クライアントからの要求が増えていて、少し不安を感じる...
4,月曜日,12:00 - 12:30,昼食,急いでランチを取る。サンドイッチを片手に、サーバー復旧作業を指示。なんとか午後のミーティング...
5,月曜日,12:30 - 16:00,仕事関連,午後はクライアントとの追加打ち合わせ。トラブルの影響で予定が押しているため、集中して対応。ト...
6,月曜日,16:00 - 17:30,帰宅,電車に乗って帰宅。今日は早めに帰り、家族と夕食の時間を確保する。帰宅途中、子供が学校で風邪を...
7,月曜日,17:30 - 18:00,家事・夕食準備,子供の様子を見つつ、簡単な夕食を準備。夫が薬を買って帰宅。みんなで夕食を取るが、子供の体調が...
8,月曜日,18:00 - 19:30,家族との時間,子供を寝かしつける。夫と今日の出来事を話し合い、明日のスケジュールを再確認。家事を手伝っても...
9,月曜日,19:30 - 21:00,リラックス,今日は疲れたので、早めにベッドに入る。読書をしながら、明日の業務の準備を少しだけ進める。


In [42]:
# 各時間帯ごとの行動種別のカウント
activity_counts = df.apply(pd.Series.value_counts).fillna(0)

# 各時間帯ごとの割合を計算
activity_ratios = activity_counts.div(activity_counts.sum(axis=0), axis=1)
calc_df = activity_ratios.reset_index().melt(id_vars=['index'], var_name='Time', value_name='Rate')
calc_df.rename(columns={'index': 'Activity'}, inplace=True)

In [8]:
calc_df["Time"] = pd.to_datetime(calc_df["Time"],format="%H:%M").dt.time
Calc_df = sort_df_by_list(calc_df,"Activity",middle)

NameError: name 'calc_df' is not defined

In [9]:
Time_table = time_table.copy()
Time_table["Time"] = pd.to_datetime(Time_table["Time"],format="%H:%M").dt.time

In [10]:
import pandas as pd
import plotly.graph_objects as go

# 各アクティビティに対して色を指定
color_map = {
    '睡眠': 'blue',
    'その他・不明': 'red',
    'マスメディア接触': 'green',
    'レジャー活動': 'purple',
    '仕事関連': 'orange',
    '休息': 'pink',
    '会話・交流': 'cyan',
    '学業': 'brown',
    '家事': 'yellow',
    '療養・静養': 'black',
    '社会参加': 'magenta',
    '身のまわりの用事': 'lightblue',
    '通勤': 'darkgreen',
    '通学': 'lightgreen',
    '食事': 'gray'
}

# Plotlyで折れ線グラフを作成
fig = go.Figure()

# time_tableの実線グラフ
for activity in Time_table['Activity'].unique():
    filtered_df = Time_table[Time_table['Activity'] == activity]
    fig.add_trace(go.Scatter(
        x=filtered_df['Time'], 
        y=filtered_df['Rate'], 
        mode='lines', 
        name=f'{activity} (実線)', 
        line=dict(dash='solid', color=color_map.get(activity))
    ))

fig.update_layout(
    title="Activityごとの時間推移",
    xaxis_title="時間",
    yaxis_title="割合",
    legend_title="活動",
    template="plotly_dark"
)

fig.show()


In [3]:
time_table

NameError: name 'time_table' is not defined